#### Function Object

In [1]:
''' A python functions is an objects of <class 'function'>. '''
def f():
    ''' This the doc string of f(). '''
    print('Hello')
print(type(f))

<class 'function'>


In [2]:
''' A 'function object' has a dict for its namespace. '''
f.__dict__

{}

In [3]:
''' A 'function object' has the following special attributes. '''
print(f.__name__)
print(f.__class__)
print(f.__doc__)
print(f.__code__)

f
<class 'function'>
 This the doc string of f(). 
<code object f at 0x000001871F0558B0, file "C:\Windows\Temp\ipykernel_22400\1456961915.py", line 2>


In [4]:
''' A function local variable holds its state during the function activation.
A 'function object' dict holds its state during the object lifetime.  '''
def f():
    x = 1
# print(dir(f))   ## 'x' is local variable and not in the dict of the function

''' We can explicitly add a variable to the function dict. '''
f.y = 2
# print(dir(f))   ## 'y' is in the function dict. 

In [5]:
''' So Python functions can be statefull.
Ex. To make a function remembers its state between calls. '''
def inc():
    if 'c' not in inc.__dict__:
        inc.c = 0
    inc.c += 1            ## inc.c is a state of inc().
    return inc.c

[inc() for _ in range(3)]

[1, 2, 3]

#### Callable Object
Python believes that using functions is more simple than objects. So Python spended a lot of
hard works for hiding object-oriented concepts under function usages.

In [6]:
''' An object of any classes is 'callable' if it has __call__().
That allows callng an object like a function. '''
class A:
    def __call__(self, name):
        print('Hello', name)

''' () is the 'call' operator.
If () is applied to a class, the meta-class's __call__() is invoked that  creates 
object of the class and be returned. '''
a = A()    ## 'a' is an object of 'A' class.

''' If () is applied to a callable object, its __call__() is invoked. '''
# a('John')                           ## Hello John

''' callable(<obj>) is a built-in function for testing if a class or object is callable. '''
(callable(A), callable(a))

(True, True)

#### First Class Object

In [7]:
''' A function is 'first class' object if:
      1. Created dynamically at runtime.
      2. Assignable to variables.
      3. Passable to/from functions.
Languages that support 'first class' object is called 'functional' languages.
Ex. Assign a function to a variable.     '''
def hello():
    print('Hello')
greet = hello   ## 'greet' is a variable that has a function as value.
greet()         ##  and it can be called as a function

Hello


In [8]:
''' Ex. Passing function as parameter. '''
def say_hello(name):
    print('Hello ' + name)
def say_hi(name):
    print('Hi ' + name)
def run(func, name):
    func(name)
run(say_hello, 'John')
run(say_hi, 'Jack')

Hello John
Hi Jack


In [9]:
''' Ex. A function that returns a function. '''
def get_hello():
    def f():
        print('Hello')
    return f
get_hello()()

Hello


In [10]:
''' An inner function can be returned to be used outside.
If a returned function refers to objects not in its scope, the object
must be copied to go with the function, this is called 'capturing'.
A function with its captured objects is called 'closure'. '''
def get_closure(name):
    msg = 'Hello '
    def f():
        return msg + name
    return f
print(get_closure('John')())

Hello John


In [11]:
''' A closure may be returned to many callers.
Most languages e.g. Java have restriction that closure state must be immutable.
But Python allows mutable closure states. '''
def get_counter():
    x = 0
    def counter():
        nonlocal x
        x += 1
        return x
    return counter
    
c = get_counter()
(c(), c())                 ## 1 2

(1, 2)

#### Fixed State Callable Object

In [12]:
''' If a function is frequently called with some fixed parameter. 
Callable objects can be created with fixed state to simplify calling the function. '''
class Greet:
        def __init__(self, msg):
            self.msg = msg

        def __call__(self, name):
            print(self.msg + ' ' + name)
            ## Returns a callable greet with fix 'msg'.

hello = Greet('Hello')
hello('John')
hi = Greet('Hi')
hi('Jack')

Hello John
Hi Jack


#### Partial functions

In [13]:
''' 'Partial functions' are created from another function with some fixed parameters.
That simplify calling the function without the fixed parameters.
The 'functools' lib has partial() for creating a partial function from an already defined function. '''
import functools
def add(a, b):
    return a + b

''' Only the right most parameters may have default values. '''
add_one = functools.partial(add, b=1)   ## 'b' parameter is fixed.
add_two = functools.partial(add, b=2)
(add_one(1), add_two(1))

(2, 3)

In [14]:
''' The 'operator' lib defines a lot of ready to use operators. '''
import operator
# print(dir(operator))

''' Ex. 'mul' is an operator for multiplication. '''
# print(operator.mul(2, 3))       ## 6

''' Ex. A partial function for multiply by 3.  '''
triple = functools.partial(operator.mul, 3)
print(triple(2))

6


#### Decorator

In [15]:
''' A 'decorator' is a function that accepts a function as parameter and returns 
a modified function as the result.

Ex. To add an extra line to the doc string of a function. '''
def written_by_john(f):
    f.__doc__ += '\nWritten by: John.'
    return f

## 1. Manually implemented decorator, (performed at runtime).
def hello():
    '''Print: Hello'''
    print('Hello')

hello = written_by_john(hello)
print(hello.__doc__)

Print: Hello
Written by: John.


In [16]:
''' Alternatively there is @<decorator> that adds the <decorator> to the function below.
It is performed when the function is created (at runtime).
There can be multiple decorators applied, the lowest is the first. '''
@written_by_john
def hi():
   '''Return: Hi'''
   return 'Hi!'
print(hi.__doc__)

Return: Hi
Written by: John.


In [17]:
''' A decorator may add parameters to the function, using function wrapper. '''
def add_param(func):
    def _f(name):
        return func() + ' ' + name
    return _f

@add_param
def hello():
    return 'Hello'

print(hello('John'))

Hello John


In [18]:
''' A decorator may has parameters by using double function wrapper. '''
def add(msg):
    def __f(func):
        def _f(name):
            return func() + ' ' + name + ', ' + msg
        return _f
    return __f

@add('how do you do?')
def hello():
    return 'Hello'

print(hello('John')) 

Hello John, how do you do?


### Lambda

In [19]:
''' 'Lambda' is a function with no names and defined as expressions(not statement).
            lambda <parameters> : <body>
A lambda expression may be invoked directly by applyed with (arguments). '''
# (lambda name: print('Hello ' + name))('John')   ##  Hello John

''' A lambda expression results a lambda which is a callerable object which 
can be used as a function and assigned to a name. But that an unhealthy approach
to define a function.  '''
hello = lambda name: print('Hello ' + name) 
hello('Jack')

Hello Jack


### Lambda Syntax

In [20]:
''' A lambda may have no parameters.
If the body is a value, it is automatically returned. '''
(lambda : 'Hello')()

'Hello'

In [21]:
''' If <body> is an expression, its result is returned.  '''
(lambda x : x+1)(1)

2

In [22]:
''' Lambda parameters are not allowed to enclosed with (). '''
# (lambda (name): 'Hello' + name)('John')   ## error
(lambda name: 'Hello ' + name)('John')

'Hello John'

In [23]:
''' Lambda parameters are separated by ','.   '''
(lambda a, b : a+b)(1,2)

3

In [24]:
''' <body> must be a single value or expression.  '''
# (lambda a, b : a + b, a * b)(1,2)  ## error

''' Wrapping with () would a tuple. '''
# print((lambda a, b : (a + b, a * b))(1,2))    ## (3, 2)

''' Wrapping with [] would a list. '''
# print((lambda a, b : [a + b, a * b])(1,2))    ## [3, 2]

''' Wrapping with {} would a set. '''
# print((lambda a, b : {a + b, a * b})(1,2))    ## {2, 3}

''' Or make it a dict. '''
# print((lambda x : { 'x': x, 'x+1': x+1 })(1))   ## {'x': 1, 'x+1': 2}

''' print() is an expression, not a statement, it returns a None. '''
(lambda: { print('Hello'), print('Hi') })()
## So enclosing <body> with {} does not make it a block.
## The separtor is a ',' not a ';'. And the result is a set.

Hello
Hi


{None}

#### Function VS Lambda

In [25]:
''' - Functions have names which can be refered to repeatedly.
Lambdas have no names, to use again if no reference the expression must be redefined.  '''
(lambda name: print('Hello', name))('Johnk') 
(lambda name: print('Hello', name))('Jack') 

Hello Johnk
Hello Jack


In [26]:
''' -Function names are keep in their namespace and stay their as lobg as needed.
So functions are good for long-term and frequently used.
Lambdas can be used without name (as an expression) and no storage in namespace.
So lambdas are mostly used once. '''
def func():
    def hello(name):
        print('Hello', name)
    hello('John')            ## Hello John
    print(dir())             ## ['hello']
# func()

def lamb():
    (lambda name: print('Hi', name))('Jack') 
    print(dir())
lamb()

Hi Jack
[]


In [27]:
''' Lambdas are frequently passed as arguments to short-life function. '''
def run(func):
    func('John')
    func('Jack')
run((lambda name: print('Hello', name)))

Hello John
Hello Jack


#### Function str
A simple less powerful alternative to lambdas.

In [28]:
''' A lambda is function that expressed as an expression.
A 'function str' is a function that expressed as a str and resulted a str.
It allows representing a function as a str.
'function str' is a str prefixed with a 'f'. '''
s = f'1+2'
# print(s, type(s))         ## 1+2 <class 'str'>

''' 'function str' allows embedded expression by enclosing with {}.
Embedded expressions are evalulated and results are inserted there. '''
a = 1; b =2
# print(f'a+b = {a+b}')      ## a+b = 3

''' Expressions may contain function calls. '''
# print(f'{len("John")}')    ## 4

''' Expressions allow indexing '''
a = ['John', 'Jack', 'Joe']
# print(f'Hi {a[1]}')            ## Hi Jack
d = { 'fname': 'John', 'lname': 'Rambo'}
# print(f'Hello {d["fname"]}')   ## Hello John

''' Expressions support format characters. '''
x = 1.23456
# print(f'x = {x:.2f}')       ## x = 1.23

### Functional Programming
Functional Programming is a programming paradigm that everything are functions, no statements, no side effect, no iterations. <p/>
<b>Aggregation</b> is a mechanism to apply a function to every elements in a sequence one by one resulting a sequence of results. That allows programming without explicit loopings.

In [29]:
''' Python supports aggregarion by a built-in function:
            map(<function>, <sequence>)
<function> may be function, lambda or callable object.
<sequence> may be collections, generator, iterable or range.
map() returns a generator. 

Ex. Computes a list of 5 square numbers. '''
sq = lambda n : n*n
list(map(sq, range(5)))   ## list() collects the results from generator.

[0, 1, 4, 9, 16]

In [30]:
''' Decision Aggregation:
            filter(<predicate>, <sequence>)
<predicate> is applied to elements in <sequence>, if True the element is passed to the result. '''

''' To check if n is a square number. '''
import math
def is_sq(n):
    return n == int(math.sqrt(n)) * int(math.sqrt(n))

''' Ex. Computes a list of square numbers under 20. '''
# print(list(filter(lambda n: is_sq(n), range(20))))     ## [0, 1, 4, 9, 16]

''' But comprehension can do the job nicely. '''
print([x for x in range(20) if is_sq(x) ])

[0, 1, 4, 9, 16]


#### Reduction

In [31]:
''' Reduction is the process of reducing a sequence into a value.
The 'functools' lib has:
             reduce(<function>, <sequence>)
<function> is applied to two successive elements in the <sequence>,
the result is accumulated to a value. '''
import functools, operator
r = range(1, 11)

''' Sumation '''
# print(functools.reduce(operator.add, r))       ## 55

''' Factorial '''
print(functools.reduce(operator.mul, r))

3628800


In [32]:
''' Map-Reduce Programming is a style of programming:
 1. Start with a source sequence that provides input values.
 2. Map/Filter the sequnce.
 3. Reduce the results to a value.

Ex. To compute summation of sqaure even numbers less than 10.
A filter predicate'''
is_even = lambda x: x % 2 == 0

''' Procedural (Imperative) '''
r1 = []
for x in range(10):
    if is_even(x):
        r1.append(sq(x))
r2 = 0
for x in r1:
   r2 += x
# print(r2)                           ## 120

''' Map-Reduce (Functional) '''
print(functools.reduce(operator.add,
                       map(sq, filter(is_even, range(10)))))

''' Comprehension '''
print(sum([sq(x) for x in range(10) if is_even(x)]))

120
120
